In [9]:
import json
import time
import logging
import os

In [39]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Hello from the logging system!")

INFO:root:Hello from the logging system!


In [15]:
#         auto repo_id_code = "def extract_repo_id(row):\n"
#                             "\tif 2012 <= row['year'] <= 2014:\n"
#                             "\t\treturn row['repository']['id']\n"
#                             "\telse:\n"
#                             "\t\treturn row['repo']['id']\n";

#         return ctx.json(pattern)
#                   .withColumn("year", UDF("lambda x: int(x['created_at'].split('-')[0])"))
#                   .withColumn("repo_id", UDF(repo_id_code))
#                   .filter(UDF("lambda x: x['type'] == 'ForkEvent'"))
#                   .selectColumns(std::vector<std::string>({"type", "repo_id", "year"}));

In [51]:


# pipeline executed per path (note row numbers with execeptions etc.)
def github_pipeline(input_path):
    
    logging.info('opening {}'.format(os.path.basename(input_path)))
    with open(input_path, 'r') as fp:
        lines = fp.readlines()
    logging.info('Parsing {} rows to dict objects...'.format(len(lines)))    
    rows = [json.loads(line) for line in lines]
    logging.info('Parsed {} rows.'.format(len(rows)))
    
    
    def extract_repo_id(row):
        if 2012 <= row['year'] <= 2014:
            return row['repository']['id']
        else:
            return row['repo']['id']
        
    def with_column(rows, name, f):
        def helper(row):
            try:
                row[name] = f(row)
                return row
            except Exception as e:
                return e
            
        return [helper(row) for row in rows]
        
    def count_exceptions(rows):
        e_rows = list(filter(lambda x: isinstance(x, Exception), rows))
        if len(e_rows) != 0:
            print(len(e_rows))
        
    # now process
    tmp1 = with_column(rows, 'year', lambda x: int(x['created_at'].split('-')[0]))
    logging.info('First UDF applied.')
    tmp2 = with_column(tmp1, 'repo_id', extract_repo_id)
    logging.info('Second UDF applied.')
    count_exceptions(tmp2)
    # how many exceptions and of what kind?
    tmp3 = list(filter(lambda x: not isinstance(x, Exception) and x['type'] == 'ForkEvent', tmp2))
    logging.info('Filtered rows down.')
    # get result
    tmp4 = list(map(lambda x: (x['type'], x['repo_id'], x['year']), tmp3))
    res = tmp4
    
    return res

In [52]:
sample_path = '../../tuplex/test/resources/hyperspecialization/github_daily/2013-10-15.json.sample'

In [55]:
rows = github_pipeline(sample_path)
print('Got as result for {} in total {} rows'.format(os.path.basename(sample_path), len(rows)))

INFO:root:opening 2013-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...
INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.


61
Got as result for 2013-10-15.json.sample in total 55 rows


In [56]:
import glob

In [58]:
paths = glob.glob('../../tuplex/test/resources/hyperspecialization/github_daily/*.json.sample')

res_dict = {}
for path in paths:
    rows = github_pipeline(path)
    print('Got as result for {} in total {} rows'.format(os.path.basename(path), len(rows)))
    res_dict[os.path.basename(path)] = len(rows)
    
print(res_dict)

INFO:root:opening 2014-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...
INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2020-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...
INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2011-10-15.json.sample


1
Got as result for 2014-10-15.json.sample in total 62 rows
Got as result for 2020-10-15.json.sample in total 24 rows


INFO:root:Parsing 1200 rows to dict objects...
INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2015-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...
INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2021-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...


45
Got as result for 2011-10-15.json.sample in total 30 rows
Got as result for 2015-10-15.json.sample in total 40 rows


INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2016-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...
INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2013-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...


...

61
Got as result for 2013-10-15.json.sample in total 55 rows
Got as result for 2017-10-15.json.sample in total 15 rows


INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2019-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...
INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.
INFO:root:opening 2018-10-15.json.sample
INFO:root:Parsing 1200 rows to dict objects...


57
Got as result for 2012-10-15.json.sample in total 53 rows
Got as result for 2019-10-15.json.sample in total 24 rows


INFO:root:Parsed 1200 rows.
INFO:root:First UDF applied.
INFO:root:Second UDF applied.
INFO:root:Filtered rows down.


Got as result for 2018-10-15.json.sample in total 26 rows
{'2014-10-15.json.sample': 62, '2020-10-15.json.sample': 24, '2011-10-15.json.sample': 30, '2015-10-15.json.sample': 40, '2021-10-15.json.sample': 18, '2016-10-15.json.sample': 31, '2013-10-15.json.sample': 55, '2017-10-15.json.sample': 15, '2012-10-15.json.sample': 53, '2019-10-15.json.sample': 24, '2018-10-15.json.sample': 26}


In [59]:
import pandas as pd

In [64]:
pd.DataFrame(list(res_dict.items()), columns=['file', 'out_row_count']).sort_values(by='file')

,file,out_row_count
2,2011-10-15.json.sample,30
8,2012-10-15.json.sample,53
6,2013-10-15.json.sample,55
0,2014-10-15.json.sample,62
3,2015-10-15.json.sample,40
5,2016-10-15.json.sample,31
7,2017-10-15.json.sample,15
10,2018-10-15.json.sample,26
9,2019-10-15.json.sample,24
1,2020-10-15.json.sample,24
